<h1>Aprendizaje automático automatizado usando Auto-Sklearn para predecir accidentes cerebrovasculares (Ejecución de tiempo reducido)</h1>

<hr>

<h3>En este Notebook se va a preparar y ejecutar aprendizaje automático automatizado sobre el <i>dataset</i> 'healthcare-dataset-stroke-data.csv' (https://www.kaggle.com/fedesoriano/stroke-prediction-dataset) durante 10 minutos, con el fin de descubrir el rendimiento de Auto-Sklearn y si es capaz de obtener buenos resultados en un tiempo de entrenamiento muy limitado.</h3>

<hr>

<h2>1. Importación de librerías y preparación del <i>dataset</i></h2>

<p>En primer lugar, se importan los paquetes y librerías necesarias para la ejecución del código:</p>

In [1]:
import autosklearn.classification as asklc
import autosklearn.metrics as asklm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

<p>A continuación, se procede a la lectura de los datos del archivo <i>healthcare-dataset-stroke-data.csv</i>. Para ello, se han visualizado los datos en bruto previamente para conocer qué atributos hay y sus respectivos tipos. Una vez resvisados, se procede a crear el <i>dataframe</i> de Pandas con los tipos de dato correctos y con el contenido del fichero CSV.</p>

In [2]:
strokes = pd.read_csv('datasets/healthcare-dataset-stroke-data.csv', dtype={'gender': 'category', 'age': float, 'hypertension': 'int8', 'heart_disease': 'int8', 'ever_married': 'category', 'work_type': 'category', 'Residence_type': 'category','avg_glucose_level': float, 'bmi': float, 'smoking_status': 'category', 'stroke': 'int8'})
strokes

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


<p>Una vez importados los datos, se procede a comprobar si se han leído correctamente mediante el siguiente código. Además, Pandas mostrará información sobre el número de filas no nulas de cada atributo. Para que el conjunto de datos sea válido para entrenar un modelo, no deberá tener ningún valor nulo.</p>

In [3]:
strokes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int8    
 4   heart_disease      5110 non-null   int8    
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                4909 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int8    
dtypes: category(5), float64(3), int64(1), int8(3)
memory usage: 200.5 KB


<p>Como se puede ver, el <i>dataset</i> tiene un total de 5110 entradas y 12 columnas. Todos los atributos tienen valor en la totalidad de las instancias, excepto la columna 9 (bmi), a la que le faltan 201 valores. Existen diferentes técnicas para solucionar este problema, tales como eliminar las filas que contengan los valores nulos, sustituirlos por un número concreto o rellenarlos con la media global de dicha propiedad. Para este caso, se ha considerado esta última solución como la más óptima. Para ello, será tan sencillo como ejecutar el siguiente código:</p>

In [4]:
strokes = strokes.fillna(strokes.mean())

<p>Comprobamos que se hayan rellenado correctamente los valores nulos:</p>

In [5]:
strokes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int8    
 4   heart_disease      5110 non-null   int8    
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                5110 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int8    
dtypes: category(5), float64(3), int64(1), int8(3)
memory usage: 200.5 KB


<p>Además de no contener valores nulos, el <i>dataset</i> deberá ser tratado para convertir los valores de los atributos categóricos en números enteros, de forma que los algoritmos de aprendizaje puedan trabajar con ellos. Para hacer esto, seleccionaremos las columnas de tipo categoría y a cada una le aplicaremos la función <i>cat.codes</i>.</p>

In [6]:
cat_columns = strokes.select_dtypes(['category']).columns
strokes[cat_columns] = strokes[cat_columns].apply(lambda x: x.cat.codes)

<p>Como se puede ver, los tipos 'category' han pasado a ser enteros, habiéndose asignado a cada valor de la categoría un número:</p>

In [7]:
strokes

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


<p>Una vez preparado el <i>dataset</i>, se puede proceder a construir el <i>dataframe</i> que contenga las características de las que extraer la información y el que almacene la variable objetivo, en este caso el atributo binario 'stroke', el cual determinará si el paciente es propenso a sufrir una isquemia cerebral o no. Para ello, del <i>dataframe</i> completo se han eliminado las columnas 'id' (puesto que no aporta información) y 'stroke' (ya que es la variable objetivo) para construir el de características, y se ha incluido únicamente la columna 'stroke' para crear el <i>dataframe</i> objetivo.</p> 

In [8]:
features = strokes.drop(['stroke', 'id'], axis=1)
target = strokes['stroke']

<p>El contenido y la estructura de los <i>dataframes</i> son las siguientes:</p>

In [9]:
features

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,67.0,0,1,1,2,1,228.69,36.600000,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2
2,1,80.0,0,1,1,2,0,105.92,32.500000,2
3,0,49.0,0,0,1,2,1,171.23,34.400000,3
4,0,79.0,1,0,1,3,0,174.12,24.000000,2
...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1


In [10]:
target

0       1
1       1
2       1
3       1
4       1
       ..
5105    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 5110, dtype: int8

<p>Otro aspecto muy importante a valorar del <i>dataset</i> es su gran desequilibrio entre los casos con accidente cerebrovascular y los pacientes no propensos a él. Como se puede ver, de 5110 casos, solo 249 son positivos. Esto provocará que si utilizamos como método de evaluación del aprendizaje una métrica como la precisión, un predictor trivial que siempre devuelva 0 (no propenso) tendrá una precisión de más del 95% con este <i>dataset</i>. Por tanto, se deberá utilizar otra métrica para evaluar la calidad del modelo de aprendizaje.</p>

In [11]:
target.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

<p>De todas las métricas posibles para evaluar modelos, se ha escogido la ROC AUC, ya que esta valora que se predigan correctamente tanto los casos positivos como los negativos, pudiendo llegar únicamente al 0,5 de precisión si se predicen sólo los casos negativos, de modo que un predictor trivial tendría un mal rendimiento. De esta manera, se premiará al modelo si es capaz también de 'aprender' y predecir los casos positivos de isquemia, aunque estos sean la minoría.</p>

<p>Como último paso previo al comienzo del entrenamiento, se deberán dividir los <i>dataframes</i> de características y el objetivo en dos partes cada uno, de manera que se tengan datos para el aprendizaje (en este caso serán el 75% de ellos) y para la prueba de precisión (el 25%)</p>

In [12]:
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, target, train_size=0.75, test_size=0.25)

<h2>2. Declaración del clasificador de aprendizaje automático automatizado y búsqueda del mejor modelo en un tiempo reducido</h2>

<p>La búsqueda del mejor modelo se hará utilizando el método <i>AutoSklearnClassifier</i>, al cual se le pasa por parámetro:
<ul>
    <li>La métrica a mejorar elegida (AUC de la curva ROC)</li>
    <li>El tiempo máximo de entrenamiento en segundos (que en este caso será la forma de detener el entrenamiento, 600 segundos, equivalente a 10 minutos)</li>
    <li>El tiempo máximo de entrenamiento de una instancia (20 segundos)</li>
    <li>El parámetro <i>n_jobs</i>, el cual indicará al programa que se utilicen todos los núcleos del procesador disponibles</li>
</ul>
Acto seguido se ejecuta el método <i>fit</i>, el cual iniciará la búsqueda y entrenamiento del mejor modelo. Este parará tras 10 minutos. Un detalle a destacar es que tendremos que pasarle al método las copias de los <i>dataframes</i>, ya que este los modificará durante el entrenamiento.</p>

In [13]:
cls = asklc.AutoSklearnClassifier(metric = asklm.roc_auc, time_left_for_this_task=600, per_run_time_limit = 20, n_jobs = -1)
cls.fit(training_features.copy(), training_target.copy(), dataset_name='stroke')

AutoSklearnClassifier(metric=roc_auc, n_jobs=-1, per_run_time_limit=20,
                      time_left_for_this_task=600)

<p>Una vez finalizado el entreamiento, la manera de ver los resultados que se han obtenido será ejecutando el siguiente código, el cual arrojará la información del <i>sprint</i>:</p>

In [14]:
print(cls.sprint_statistics())

auto-sklearn results:
  Dataset name: stroke
  Metric: roc_auc
  Best validation score: 0.858252
  Number of target algorithm runs: 152
  Number of successful target algorithm runs: 135
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 16
  Number of target algorithms that exceeded the memory limit: 0



<p>A continuación se muestra la precisión AUC del mejor modelo obtenido prediciendo los datos de prueba:</p>

In [15]:
testing_proba = cls.predict_proba(testing_features)[:,1]
print("Puntuación de AUC prediciendo los datos de prueba: ", sklearn.metrics.roc_auc_score(testing_target, testing_proba))

Puntuación de AUC prediciendo los datos de prueba:  0.8201359825363941
